In [ ]:
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

# Custom Import
from vectorstore_logic import check_for_updates_to_rules, create_or_load_vectorstore_optcg_rulebooks

_ = load_dotenv() # Loads the .env file - e.g. the OPENAI_API_KEY

In [ ]:
vectorstore = create_or_load_vectorstore_optcg_rulebooks()
check_for_updates_to_rules()

Documents have not changed since last vector store creation.
Existing hash found: True, Document changes detected: False
No updates needed.


False

In [25]:
## Test the vector store
print("Testing vector store with a sample query...")
sample_query = "How many cards can I draw on my first turn?"
results = vectorstore.similarity_search(sample_query, k=3)

[result.metadata["source"] + str(result.metadata["page_label"]) + ": " + result.page_content for result in results]

Testing vector store with a sample query...


['comprehensive_rules12: their effects first, in any order, followed by the player who did not choose \nto go first or second, who then processes their effects in any order. \n5-2-1-5-2. If changes are made to a deck as the result of an effect that reads “At the \nstart of the game”, it is then shuffled by the owner of that deck. \n5-2-1-6. Each player draws 5 cards from their deck  as their opening hand. Then, \nbeginning with the player going first, each player may redraw their hand once \naccording to the procedure below.  \n5-2-1-6-1. The player returns all of the cards in their hand to their deck, reshuffles, \nand then redraws 5 cards. \n5-2-1-7. Each player places a number of cards from the top of their deck equal to the \nLife value of their Leader face-down in their Life area such that the card at the \ntop of their deck is at the bottom in their Life area. \n5-2-1-8. The first player begins the game and starts their turn.',
 'comprehensive_rules12: cannot contain a specified 

In [ ]:
sample_query = "What is considered private knowledge?"
results = vectorstore.similarity_search(sample_query, k=3)

[result.metadata["source"] + str(result.metadata["page_label"]) + ": " + result.page_content for result in results]

['tournament_rules21: opponent should receive a Warning. \n• Irreparable game states should not result in restarting a game or match. \n• If a player has damaged the game state due to an illegal action or missed auto, and \ngameplay can be rewound to the point of that illegal action, it will be rewound even if the \nopponent would gain an advantage from information that was previously Private \nKnowledge. \n4.11 Public Knowledge \nThe following information is considered to be public knowledge, unless a card effect states \notherwise: \n• The number of cards in a player’s hand; \n• The number of cards in a player’s deck (this includes main and DON!! deck); \n• The number and names of cards in a player’s trash area; \n• The number of Life cards remaining; \n• Information printed on a specific card mentioned by name or otherwise clearly described; \n• Which cards have been played during the current turn. \nPlayers must answer questions involving the above topics truthfully. Players may no

In [ ]:
sample_query = "What is considered public knowledge?"
results = vectorstore.similarity_search(sample_query, k=3)

[result.metadata["source"] + str(result.metadata["page_label"]) + ": " + result.page_content for result in results]

['tournament_rules21: opponent should receive a Warning. \n• Irreparable game states should not result in restarting a game or match. \n• If a player has damaged the game state due to an illegal action or missed auto, and \ngameplay can be rewound to the point of that illegal action, it will be rewound even if the \nopponent would gain an advantage from information that was previously Private \nKnowledge. \n4.11 Public Knowledge \nThe following information is considered to be public knowledge, unless a card effect states \notherwise: \n• The number of cards in a player’s hand; \n• The number of cards in a player’s deck (this includes main and DON!! deck); \n• The number and names of cards in a player’s trash area; \n• The number of Life cards remaining; \n• Information printed on a specific card mentioned by name or otherwise clearly described; \n• Which cards have been played during the current turn. \nPlayers must answer questions involving the above topics truthfully. Players may no

In [3]:
delete_vectorstore_optcg_rulebooks()

Deleted vector store at C:\Users\tyson\.cache\optcg_rulebooks_vectorstore


In [ ]:
@tool
def rulebook_lookup(query: str) -> str:
    """Looks up a rule in the One Piece TCG rulebook."""
    # In a real implementation, this would query a database or API.
    rules = {
        "What happens if two characters with the same name are played on the same team?": "Characters with the same name cannot be played on the same team. If you already have a character in play, you cannot play another character with the same name.",
        "How does the Don!! system work?": "Don!! cards are used to pay costs and activate abilities. You can attach Don!! cards to characters to increase their power or use them to pay for events and character abilities.",
        "What is the difference between active and rest positions?": "Active position means the card is upright and can attack or use abilities. Rest position means the card is turned sideways and cannot attack until it becomes active again during your next turn.",
        "How do you win the game?": "You win by reducing your opponent's life to 0. Life is reduced when your opponent takes damage and has no cards left in their life area to trash.",
        "What is a counter ability?": "Counter abilities can be activated during your opponent's turn when specific conditions are met, usually when one of your characters is being attacked."
    }
    return rules.get(query, "Rule not found.")

In [ ]:
tools = [rulebook_lookup]
agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4.1-mini", temperature=0),
    name="RulebookAgent",
    tools=tools, 
    prompt="You are a helpful assistant that helps people find information in the Rulebook for One Piece TCG. You have access to the following tools: {tools}. Use them to find the information the user is looking for. If you don't know the answer, just say you don't know. Do not try to make up an answer.",
)

In [ ]:
response = agent.invoke(
    {"messages": [{
        "role": "user", 
        "content": "What happens if two characters with the same name are played on the same team?"
    }]}
)
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

What happens if two characters with the same name are played on the same team?
================================== Ai Message ==================================
Name: RulebookAgent
Tool Calls:
  rulebook_lookup (call_PKkircNK3uQSxN1NV6lDwEsx)
 Call ID: call_PKkircNK3uQSxN1NV6lDwEsx
  Args:
    query: two characters with the same name on the same team
================================= Tool Message =================================
Name: rulebook_lookup

Rule not found.
================================== Ai Message ==================================
Name: RulebookAgent

I couldn't find a specific rule in the One Piece TCG rulebook about what happens if two characters with the same name are played on the same team. If you have any other questions or need information on a related topic, feel free to ask!


In [ ]:
rules = {
        "What happens if two characters with the same name are played on the same team?": "Characters with the same name cannot be played on the same team. If you already have a character in play, you cannot play another character with the same name."}
rules.get("What happens if two characters with the same name are played on the same team?", "Rule not found.")

'Characters with the same name cannot be played on the same team. If you already have a character in play, you cannot play another character with the same name.'

In [ ]:
# Example: Load a PDF from a URL
# Replace this with your actual PDF URL
pdf_url = "https://example.com/your-pdf-file.pdf"

# Uncomment the lines below to test with a real PDF URL
# documents = load_pdf_from_url(pdf_url)
# if documents:
#     print(f"Successfully loaded {len(documents)} pages from the PDF")
#     
#     # Show first page content (first 500 characters)
#     if len(documents) > 0:
#         print(f"\nFirst page content preview:")
#         print(documents[0].page_content[:500] + "...")
#         
#         # Show metadata
#         print(f"\nPage metadata:")
#         print(documents[0].metadata)
# else:
#     print("Failed to load PDF")

In [ ]:
# Debug cell - check Path object
print(f"Path type: {type(Path)}")
print(f"Path.home() type: {type(Path.home())}")
print(f"Path.home() value: {Path.home()}")

# Test the problematic line step by step
home = Path.home()
print(f"home: {home}, type: {type(home)}")

cache = home / ".cache"
print(f"cache: {cache}, type: {type(cache)}")

final_path = cache / "onepiece_vectorstore"
print(f"final_path: {final_path}, type: {type(final_path)}")

Path type: <class 'type'>
Path.home() type: <class 'pathlib._local.WindowsPath'>
Path.home() value: C:\Users\tyson
home: C:\Users\tyson, type: <class 'pathlib._local.WindowsPath'>
cache: C:\Users\tyson\.cache, type: <class 'pathlib._local.WindowsPath'>
final_path: C:\Users\tyson\.cache\onepiece_vectorstore, type: <class 'pathlib._local.WindowsPath'>
